In [16]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

#open a random image and check its size
im = Image.open("/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/hippopotamus/hippopotamus_10003.jpg")
#display.display(Image.open(str('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/hippopotamus/hippopotamus_10003.jpg')))
width, height = im.size 
print(width,height)

#resizing
im=im.resize((448,448),resample=0)
im.show()


1024 684


To load the files as a tf.data.Dataset first create a dataset of the file paths:

In [2]:
#source: https://www.tensorflow.org/tutorials/load_data/images
def image_paths(dataset_path):
    """
    input: dpath of the whole image dataset
    returns: all of the imagepaths and all of the labels/classes
    as type tensorflow.python.framework.ops.EagerTensor
    """
    
    imagepaths=[]
    #os.walk returns a tuple of three elements: (root_dir_path, sub_dirs, files)
    
    #picking up the names of the subfiles equals the names of the classes 
    classes = os.walk(dataset_path).__next__()[1]
    
    # List each sub-directory (the classes)
    for c in classes:
        #seperate animal folder(=class)
        c_dir = os.path.join(dataset_path, c)
        #make a walk object for each folder
        walk = os.walk(c_dir).__next__()
        # Add each image (walk[2]) to the imagepaths list
        for sample in walk[2]:
            imagepath = os.path.join(c_dir,sample)
            imagepaths.append(imagepath)
            
            
    #cpnvert to tensfor  (No, not yet!)     
    #imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    
    
    return imagepaths     
        

In [3]:
imagepaths= image_paths('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages')  

Write a short pure-tensorflow function that converts a file paths to an (image_data, label) pair:

In [4]:
#source: https://www.tensorflow.org/tutorials/load_data/images
#get the label of one image

def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, '/')
  # The second to last is the class-directory
  return parts[-2] 

#get_label('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/hippopotamus/hippopotamus_10003.jpg')

In [11]:
#source: https://www.tensorflow.org/tutorials/load_data/images
#decode one image

[IMG_WIDTH, IMG_HEIGHT]=[448,448]
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [12]:
##source: https://www.tensorflow.org/tutorials/load_data/images
#return img,label for one image

def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

#process_path('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/hippopotamus/hippopotamus_10003.jpg')

In [13]:
# source: https://www.tensorflow.org/tutorials/load_data/images
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
list_ds = tf.data.Dataset.list_files(str('/Users/stella/Downloads/Animals_with_Attributes2_3/JPEGImages/*/*'))
labeled_ds = list_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
labeled_ds

<ParallelMapDataset shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>

In [14]:
labeled_ds

<ParallelMapDataset shapes: ((448, 448, 3), ()), types: (tf.float32, tf.string)>

In [15]:
# source: https://www.tensorflow.org/tutorials/load_data/images
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

Image shape:  (448, 448, 3)
Label:  b'giant+panda'
